In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from math import sin, cos, sqrt, atan2
from sklearn import svm,metrics,tree,preprocessing,linear_model
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,mean_squared_error,recall_score,confusion_matrix,f1_score
df = pd.read_csv('https://raw.githubusercontent.com/weijuinlee/datathon/main/BigSupplyCo_Data_Files%20/geolocateddataset.csv?token=GHSAT0AAAAAABMK6536E2YPUFJAZVX3NG74YRTMEMA',header= 0,encoding='unicode_escape')

df.apply(lambda x: sum(x.isnull())) 
df = df.replace(np.nan, 0, regex=True)
df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'])
df['order date (DateOrders)'].dt.to_period('M')
df['order date (DateOrders)'].dt.strftime('%m/%Y')
df['yyyy'] = pd.to_datetime(df['order date (DateOrders)']).dt.year
df['mm'] = pd.to_datetime(df['order date (DateOrders)']).dt.month

R = 6373.0
lat1 = 18.2208
lon1 = 66.5901
all_distance = []
all_difference = []

for i in range(len(df)) :

  lat2 = abs(float(df['lat'][i]))
  lon2 = abs(float(df['long'][i]))
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
  c = 2 * atan2(sqrt(a), sqrt(1-a))
  distance = R * c
  all_distance.append(distance)
  difference = df['Days for shipping (real)'][i] - df['Days for shipment (scheduled)'][i]
  all_difference.append(difference)

df['Delivery Difference'] = all_difference
df['Distance'] = all_distance

df = df.drop(['Product Status','Order Zipcode','Customer Zipcode','Customer Zipcode','Sales','Product Name','Order Zipcode','Unnamed: 0','Order Item Quantity','Order Item Discount','Product Card Id', 'order date (DateOrders)'], axis=1)

training_data = df.copy()

learning = preprocessing.LabelEncoder()

training_data['Customer Segment']  = learning.fit_transform(training_data['Customer Segment'])
training_data['Customer State']    = learning.fit_transform(training_data['Customer State'])
training_data['Order Region']      = learning.fit_transform(training_data['Order Region'])
training_data['Order City']        = learning.fit_transform(training_data['Order City'])
training_data['Customer Country']  = learning.fit_transform(training_data['Customer Country'])
training_data['Market']            = learning.fit_transform(training_data['Market'])
training_data['Type']              = learning.fit_transform(training_data['Type'])
training_data['Order Status']      = learning.fit_transform(training_data['Order Status'])
training_data['Delivery Status']   = learning.fit_transform(training_data['Delivery Status'])
training_data['Category Name']     = learning.fit_transform(training_data['Category Name'])
training_data['Customer City']     = learning.fit_transform(training_data['Customer City'])
training_data['Order State']       = learning.fit_transform(training_data['Order State'])
training_data['Order Country']     = learning.fit_transform(training_data['Order Country'])

training_data.head()

xl=training_data.loc[:, training_data.columns != 'Late Delivery Risk']
yl=training_data['Late Delivery Risk']
xl_train, xl_test,yl_train,yl_test = train_test_split(xl,yl,test_size = 0.2, random_state = 42)

sc = StandardScaler()
xl_train=sc.fit_transform(xl_train)
xl_test=sc.transform(xl_test)

def classifiermodel(model_l,xl_train, xl_test,yl_train,yl_test):
  
    model_l=model_l.fit(xl_train,yl_train) 
    yl_pred=model_l.predict(xl_test)  
    accuracy_l=accuracy_score(yl_pred, yl_test) 
    recall_l=recall_score(yl_pred, yl_test)
    conf_l=confusion_matrix(yl_test, yl_pred)
    f1_l=f1_score(yl_test, yl_pred)

    print('Accuracy:', (accuracy_l)*100,'%')
    print('Recall:', (recall_l)*100,'%')
    print('Confusion Matrix: \n',(conf_l))
    print('F1:', (f1_l)*100,'%')

model_l = LogisticRegression(solver='lbfgs',random_state=0) 

classifiermodel(model_l,xl_train, xl_test,yl_train,yl_test)

model_l = KNeighborsClassifier(n_neighbors=1)
classifiermodel(model_l,xl_train, xl_test,yl_train,yl_test)

model_l = LinearDiscriminantAnalysis()
classifiermodel(model_l,xl_train, xl_test,yl_train,yl_test)